In [1]:
import numpy as np
import pandas as pd
import sys, os
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from configs.config import SET_SEED, SEED, DATASET_PATH
# from descriptor_selection.utils import MolecularFeatureExtractor
import deepchem as dc

c:\Programming\Anaconda\envs\chem2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Programming\Anaconda\envs\chem2\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [2]:


# 재현성 난수 고정
SET_SEED()

df = pd.read_csv(DATASET_PATH + '.csv')
df

,smiles,freesolv
0,CN(C)C(=O)c1ccc(cc1)OC,-11.01
1,CS(=O)(=O)Cl,-4.87
2,CC(C)C=C,1.83
3,CCc1cnccn1,-5.45
4,CCCCCCCO,-4.21
...,...,...
637,CCCCCCCC(=O)OC,-2.04
638,C1CCNC1,-5.48
639,c1cc(ccc1C=O)O,-8.83
640,CCCCCCCCl,0.29


In [4]:
Xs = np.zeros(len(df))
Ys = df['freesolv']

In [5]:
dataset = dc.data.DiskDataset.from_numpy(X=Xs, y=Ys, w=np.zeros(len(df)),ids=df['smiles'])

In [6]:
scaffoldsplitter = dc.splits.ScaffoldSplitter()

train,test = scaffoldsplitter.train_test_split(dataset)
train

<DiskDataset X.shape: (513,), y.shape: (513,), w.shape: (513,), ids: ['CS(=O)(=O)Cl' 'CC(C)C=C' 'CCCCCCCO' ... 'C1CCC(=O)CC1'
 'C[C@@H]1CC[C@H](C(=O)C1)C(C)C' 'C[C@@H]1CC[C@H](CC1=O)C(=C)C'], task_names: [0]>

In [7]:
test

<DiskDataset X.shape: (129,), y.shape: (129,), w.shape: (129,), ids: ['c1ccc2cc(ccc2c1)O' 'Cc1ccc2cc(ccc2c1)C' 'c1ccc2ccccc2c1' ...
 'Cc1cccc(c1C)Nc2ccccc2C(=O)O' 'C1CCCC(CC1)O' 'c1ccc2c(c1)CCC2'], task_names: [0]>

In [8]:
train, test = scaffoldsplitter.train_test_split(dataset, frac_train=0.8, seed=100)


In [9]:
train.ids

array(['CS(=O)(=O)Cl', 'CC(C)C=C', 'CCCCCCCO', 'CC(C)C(C)C',
       'CCCC(C)(C)O', 'CC[C@H](C)O', 'C(Br)Br', 'CC[C@H](C(C)C)O',
       'CCCCC(=O)OCC', 'CC(=CCC/C(=C\\CO)/C)C', 'CCCC(C)(C)C',
       'CC(=O)OCCOC(=O)C', 'CCOP(=S)(OCC)SCSP(=S)(OCC)OCC', 'CCCCC#N',
       'CC(C)(C)O', 'CC(C)C(=O)C(C)C', 'CCC=O', 'CN(C)C=O', 'C=CCC=C',
       'CCNCC', 'CC(C)CCOC=O', 'CCCCCCCCCCO', 'CCC(=O)OCC', 'CCCCCCCCC',
       'CC(=O)NC', 'CCCCCCCC=C',
       'C([C@H]([C@H]([C@@H]([C@@H](CO)O)O)O)O)O', 'CCCC(=O)OC',
       'CCCNCCC', 'C(F)(F)(F)F', 'CC[C@@H](C)CO', 'CCC#C', 'NN',
       'CCCCCCCCN', 'N', 'CC(F)F', 'C=CC=C', 'CN(C)C', 'CCCCCC(=O)N',
       'CC(C)CO[N+](=O)[O-]', 'C(CO[N+](=O)[O-])O', 'CCCCCCC(=O)C', 'CCN',
       'CC(Cl)Cl', 'CCCCCCCCBr', 'CCCC', 'CCl', 'CC(C)CBr', 'CC(C)SC(C)C',
       'CCCCCCC', 'C(I)I', 'CCCN(CCC)C(=O)SCCC', 'C[N+](=O)[O-]', 'CCOC',
       'COC(CCl)(OC)OC', 'CC(C)C', 'CC(C)CC(=O)O', 'CCCCl', 'CCCSCCC',
       'CCC[C@H](CC)O', 'CC#N', '[C@@H](C(F)(F)F)(OC(F)F)Cl', 'C=C

In [10]:
# 1. 먼저 SMILES → target 매핑 생성 (train set)
train_smiles = train.ids
train_targets = train.y.flatten()
train_dict = dict(zip(train_smiles, train_targets))

train_dict

{'CS(=O)(=O)Cl': -4.87,
 'CC(C)C=C': 1.83,
 'CCCCCCCO': -4.21,
 'CC(C)C(C)C': 2.34,
 'CCCC(C)(C)O': -3.92,
 'CC[C@H](C)O': -4.62,
 'C(Br)Br': -1.96,
 'CC[C@H](C(C)C)O': -3.88,
 'CCCCC(=O)OCC': -2.49,
 'CC(=CCC/C(=C\\CO)/C)C': -4.78,
 'CCCC(C)(C)C': 2.88,
 'CC(=O)OCCOC(=O)C': -6.34,
 'CCOP(=S)(OCC)SCSP(=S)(OCC)OCC': -6.1,
 'CCCCC#N': -3.52,
 'CC(C)(C)O': -4.47,
 'CC(C)C(=O)C(C)C': -2.74,
 'CCC=O': -3.43,
 'CN(C)C=O': -7.81,
 'C=CCC=C': 0.93,
 'CCNCC': -4.07,
 'CC(C)CCOC=O': -2.13,
 'CCCCCCCCCCO': -3.64,
 'CCC(=O)OCC': -2.68,
 'CCCCCCCCC': 3.13,
 'CC(=O)NC': -10.0,
 'CCCCCCCC=C': 2.06,
 'C([C@H]([C@H]([C@@H]([C@@H](CO)O)O)O)O)O': -23.62,
 'CCCC(=O)OC': -2.83,
 'CCCNCCC': -3.65,
 'C(F)(F)(F)F': 3.12,
 'CC[C@@H](C)CO': -4.42,
 'CCC#C': -0.16,
 'NN': -9.3,
 'CCCCCCCCN': -3.65,
 'N': -4.29,
 'CC(F)F': -0.11,
 'C=CC=C': 0.56,
 'CN(C)C': -3.2,
 'CCCCCC(=O)N': -9.31,
 'CC(C)CO[N+](=O)[O-]': -1.88,
 'C(CO[N+](=O)[O-])O': -8.18,
 'CCCCCCC(=O)C': -2.88,
 'CCN': -4.5,
 'CC(Cl)Cl': -0.84,
 'CCCCCCCC

In [11]:
y_ISIS = df['freesolv']
y_ISIS

0     -11.01
1      -4.87
2       1.83
3      -5.45
4      -4.21
       ...  
637    -2.04
638    -5.48
639    -8.83
640     0.29
641    -5.06
Name: freesolv, Length: 642, dtype: float64

In [12]:
train_idx = df.index[df.apply(lambda row: train_dict.get(row['smiles']) == y_ISIS.loc[row.name], axis=1)]

In [21]:
y_ISIS.loc['CN(C)C(=O)c1ccc(cc1)OC']

KeyError: 'CN(C)C(=O)c1ccc(cc1)OC'

In [ ]:
X_ISIS.index.difference(train_idx)

,smiles,freesolv
0,CN(C)C(=O)c1ccc(cc1)OC,-11.01
1,CS(=O)(=O)Cl,-4.87
2,CC(C)C=C,1.83
3,CCc1cnccn1,-5.45
4,CCCCCCCO,-4.21
...,...,...
637,CCCCCCCC(=O)OC,-2.04
638,C1CCNC1,-5.48
639,c1cc(ccc1C=O)O,-8.83
640,CCCCCCCCl,0.29


In [17]:
(lambda row: train_dict.get(row['smiles']) == y_ISIS.loc[row.name])

<function __main__.<lambda>(row)>

In [13]:
train_idx

Index([  0,   1,   2,   4,   5,   6,   7,   9,  10,  11,
       ...
       627, 629, 630, 631, 633, 634, 635, 637, 639, 640],
      dtype='int64', length=513)

In [14]:
df.loc[train_idx]

,smiles,freesolv
0,CN(C)C(=O)c1ccc(cc1)OC,-11.01
1,CS(=O)(=O)Cl,-4.87
2,CC(C)C=C,1.83
4,CCCCCCCO,-4.21
5,Cc1cc(cc(c1)O)C,-6.27
...,...,...
634,CCCCC/C=C/C=O,-3.43
635,CCCCCCC#C,0.71
637,CCCCCCCC(=O)OC,-2.04
639,c1cc(ccc1C=O)O,-8.83
